In [1]:
import numpy as np 
np.bool8 = np.bool
from plotly import express as px 
import pandas
import dghf
import test
import timeit

In [2]:
def error_values(kw_fit_arr,x_y_kw,bounds=None,time_repeats=0):
    errors = []
    times = []
    for kw_params in kw_fit_arr:
        for i,(x,y,kw) in enumerate(x_y_kw):
            _f_callable = lambda : dghf.fit(x,y,bounds=bounds,**kw_params)
            kw_fit = _f_callable()
            if time_repeats > 0:
                time_avg = timeit.timeit(_f_callable,number=time_repeats)
            else:
                time_avg = np.nan
            times.append({"Parameter set":i,"Time (s)":time_avg,**kw_params})
            for k,v in kw_fit.items():
                v_expected = kw[k]
                v_calculate = v 
                errors.append({"Parameter set":i,"Parameter":k,"Error (%)": 100 * abs((v_expected-v_calculate)/(v_expected)),**kw_params})
    return pandas.DataFrame(errors), pandas.DataFrame(times)

In [3]:
simulated_data = test.MyTestCase().simulated_data
# ignore the all nan data set
x_y_kw = [s[0] for s in simulated_data if not set(s[0][-1].values()) == set([np.nan])]
# all of the simulated data has positive hill coefficient
bounds = [[None,None],[None,None],[None,None],[0,np.inf]]

In [4]:
df_error_N, df_time_N = error_values(kw_fit_arr=[dict(coarse_n=coarse_n) for coarse_n in [1,2,3,4,5,6,7,8,9,10]],x_y_kw=x_y_kw,bounds=bounds,
                                    time_repeats=3)

In [5]:
df_error_median_N = df_error_N[["Error (%)","Parameter set","coarse_n"]].groupby(["Parameter set","coarse_n"]).median().reset_index()
df_time_and_error = df_error_median_N.merge(df_time_N,on=["Parameter set","coarse_n"])
fig = px.box(x="coarse_n",y="Error (%)",points="all",data_frame=df_time_and_error,log_y=True,width=600,height=400)
fig.show(renderer="iframe")

In [6]:
fig = px.box(df_time_N,x="coarse_n",y="Time (s)",points="all",width=600,height=400)
fig.show(renderer="iframe")

In [7]:
df_error_method, _ = error_values(kw_fit_arr=[dict(method='Powell'),dict(method='L-BFGS-B')],x_y_kw=x_y_kw,bounds=bounds)

In [8]:
fig = px.box(df_error_method,x="method",y="Error (%)",points="all",width=600,height=300,log_y=True)
fig.show(renderer="iframe")